# Тест: Параметрические критерии

In [5]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
from sklearn import cross_validation, linear_model, metrics

#### Задача 1

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [81]:
p1 = 10.
n1 = 34

p2 = 4.
n2 = 16

p1 = float(p1/n1)
p2 = float(p2/n2)

In [82]:
# Довертильеный интервал
def proportions_diff_confint_ind(p1, p2, n1, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.) # расчет z статистики 1-альфа/2
        
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [83]:
# Z статистика
def proportions_diff_z_stat_ind(p1, p2, n1, n2):

    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [84]:
# Расчет P-value по полученной Z-статистики и вида альернативы (двухсторонняя) на основе функции распределения
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided': # двухсторонняя
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less': # меньше
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater': # больше
        return 1 - scipy.stats.norm.cdf(z_stat)

In [85]:
print "95%% доверительный интервал: [%f, %f]" % proportions_diff_confint_ind(p1, p2, n1, n2)

95% доверительный интервал: [-0.217558, 0.305793]


In [86]:
print "p-value двухсторонний: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(p1, p2, n1, n2), 'greater')

p-value двухсторонний: 0.372930


Ответ: Нулевая гипотеза о том, что доли в выборках одинаковы (p1=p2), не опровергнута. Тогда эксперимент о заразительной зевоте подтвержден

#### Задача 2
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X1,X2,X3;
логистическая регрессия по признакам X4,X5,X6.
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [87]:
data = pd.read_csv('banknotes.txt', sep = '\t')

In [88]:
data.head(3)

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1


In [89]:
X = data[['X1','X2','X3','X4','X5','X6']]
Y = data.real

In [90]:
train_x, test_x, train_y, test_y = cross_validation.train_test_split(X, Y, test_size = 50, random_state = 1)

In [91]:
# Логичтическая Регрессия
log_regressor_1 = linear_model.LogisticRegression() #создание модели 1
log_regressor_2 = linear_model.LogisticRegression() #создание модели 2

log_regressor_1.fit(train_x[['X1','X2','X3']], train_y)
log_regressor_2.fit(train_x[['X4','X5','X6']], train_y)

lr_predictions_1 = log_regressor_1.predict(test_x[['X1','X2','X3']])
lr_predictions_2 = log_regressor_2.predict(test_x[['X4','X5','X6']])

print "первая модель:", metrics.accuracy_score(test_y, lr_predictions_1)
print "вторая модель:", metrics.accuracy_score(test_y, lr_predictions_2)

первая модель: 0.8
вторая модель: 0.98


Проверка нулевой гипотезы о том, что доли ошибочных предсказаний равны

In [97]:
# составим доли неправильных предсказаний
X1 = test_y != lr_predictions_1
X2 = test_y != lr_predictions_2

In [98]:
# Довертильеный интервал
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [99]:
# Z статистика
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [100]:
print "95%% доверительный интервал: [%f, %f]" % proportions_diff_confint_rel(X1, X2)

95% доверительный интервал: [0.059945, 0.300055]


In [101]:
print "p-value двухсторонний: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(X1, X2))

p-value двухсторонний: 0.003297


#### Задача 3
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [163]:
mu_0 = 525.
sigma = 100.

n = 100
mean = 541.4

In [164]:
Z = (mean-mu_0)/(sigma/np.sqrt(n))

In [165]:
Z

1.6399999999999977

In [166]:
# уровень значимости для односторонней альтернативы для больших значнеий (mu > mu_0)
1-stats.norm.cdf(Z)

0.050502583474103968

Нулевая гипотеза не отвергается, программа эффективна (полученный средний балл выше истинного среднего)

#### Задача 3.2
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [167]:
n = 100
mean = 541.5

In [168]:
Z = (mean-mu_0)/(sigma/np.sqrt(n))

In [169]:
Z

1.6499999999999999

In [170]:
# уровень значимости для односторонней альтернативы для больших значнеий (mu > mu_0)
1-stats.norm.cdf(Z)

0.049471468033648103